# Importaciones y configuracion basica

In [ ]:
import matplotlib.pyplot as plt
import re
from collections import defaultdict
import seaborn as sns
import numpy as np

sns.set_theme(style='darkgrid')

palette1 = sns.color_palette("flare")
palette2 = sns.color_palette("tab20")

RECORD_TIME = 0
RECORD_VALUE = 1

DATA_FOLDER = "data/"
GRAPHICS_FOLDER = "graficas/"
P1_FOLDER = "Parte 1/"
P2_FOLDER = "Parte 2/"

SAVE_GRAPHICS = False

# Parte 1 - Caso 1
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c1.vec"

# Parte 1 - Caso 2
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-1.vec"
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-1_25.vec"
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-1_5.vec"
FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-2.vec"
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-3.vec"
# FILE_VEC = "data/omnet-logs/Parte 1/p1_c2-4.vec"

# Parte 2 - Caso 1
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c1.vec"

# Parte 2 - Caso 2
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-1.vec"
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-1_25.vec"
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-1_5.vec"
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-2.vec"
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-3.vec"
# FILE_VEC = "data/omnet-logs/Parte 2/p2_c2-4.vec"

# Parseo

In [ ]:
metricas = set()

def extraer_datos_nodos(lines):
    vector_info = {}  # vector_id -> (nodo, categoria, subclave, metric)
    vector_data = defaultdict(lambda: ([], []))  # vector_id -> (list of record time, list of record values)

    # Expresiones regulares
    vector_lnk_re = re.compile(r'vector (\d+) Network\.node\[(\d+)\]\.lnk\[(\d+)\] "(.+?)" ETV')
    vector_app_re = re.compile(r'vector (\d+) Network\.node\[(\d+)\]\.app\s+"?([\w\s]+)"?\s+ETV')
    data_re = re.compile(r'^(\d+)\s+\d+\s+([\d.]+)\s+([\d.]+)')

    # Paso 1: Parsear definiciones de vector
    for line in lines:
        line = line.strip()
        match = vector_lnk_re.match(line)
        if match:
            vec_id, nodo_id, lnk_id, metric = match.groups()
            vector_info[int(vec_id)] = (int(nodo_id), 'lnk', int(lnk_id), metric)
            continue
        match = vector_app_re.match(line)
        if match:
            vec_id, nodo_id, metric = match.groups()
            vector_info[int(vec_id)] = (int(nodo_id), 'app', None, metric)

    # Paso 2: Parsear líneas con datos
    for line in lines:
        match = data_re.match(line.strip())
        if match:
            # match.group()
            # vector_id, n_evento, time, record_value

            vec_id, record_time, record_value = int(match.group(1)), float(match.group(2)), float(match.group(3))
            vector_data[vec_id][RECORD_TIME].append(record_time)
            vector_data[vec_id][RECORD_VALUE].append(record_value)

    # Paso 3: Construir estructura por nodo
    datos_nodos = defaultdict(lambda: defaultdict(
        lambda: defaultdict(lambda: ([], []))))

    for vec_id, (nodo_id, categoria, subclave, metric) in vector_info.items():
        if vec_id not in vector_data:
            continue
        metricas.add(metric)

        nodo_key = f"Nodo {nodo_id}"
        if categoria == 'lnk':
            cat_key = f"Lnk {subclave}"
        else:  # app
            cat_key = "App"

        datos_nodos[nodo_key][cat_key][metric][RECORD_TIME].extend(vector_data[vec_id][RECORD_TIME])
        datos_nodos[nodo_key][cat_key][metric][RECORD_VALUE].extend(vector_data[vec_id][RECORD_VALUE])

    return datos_nodos

# Extraer los datos a un diccionario

In [ ]:
# Abrir y leer el archivo
with open(FILE_VEC, "r") as f:
    lines = f.readlines()

# Extraer datos de los nodos
datos_nodos = extraer_datos_nodos(lines)

In [ ]:
# print(f"Nodos -> {sorted(datos_nodos.keys(), key=lambda x: int(x.split()[1]))}")
# print(datos_nodos['Nodo 5']["Lnk 1"])
# print(datos_nodos['Nodo 5']["App"]["Delay Source 0"])

# print(sorted(list(metricas)))

In [ ]:
# Imprimir resultados (truncado para verificar)
def print_data():
    nodos_ord = sorted(datos_nodos.keys(), key=lambda x: int(x.split()[1]))
    for nodo in nodos_ord:
        print_nodo(nodo)


def print_nodo(nodo: str):
    print(f"{nodo}:")

    grupos_ord = sorted(datos_nodos[nodo].keys())
    for grupo in grupos_ord:
        print(f"\t{grupo}:")

        metricas_ord = sorted(datos_nodos[nodo][grupo].keys())
        for metrica in metricas_ord:
            valores = datos_nodos[nodo][grupo][metrica]
            c_print = 3

            print(f"\t\t{metrica}:")
            print(
                f"\t\t\t({", ".join(map(lambda x: str(x), valores[RECORD_TIME][:c_print]))}, ...,",
                f"{", ".join(map(lambda x: str(x), valores[RECORD_TIME][-c_print:]))}",
                f"; {", ".join(map(lambda x: str(x), valores[RECORD_VALUE][:c_print]))}, ...,",
                f"{", ".join(map(lambda x: str(x), valores[RECORD_VALUE][-c_print:]))})"
                )
            print(f"\t\t\tTotal: ({len(valores[RECORD_TIME])}, {len(valores[RECORD_VALUE])}) valores")


# print_data()
# print_nodo("Nodo 0")
# print_nodo("Nodo 5")

## Utilidades

In [ ]:
def get_simulation_name():
    match = re.search(
        r"Parte (\d)\/p(\d)_c(\d)(?:-(\d(?:_\d+)?))?\.vec", FILE_VEC)
    if not match:
        return "Nombre de simulación no encontrado"

    parte = match.group(1)
    caso = match.group(3)
    inter_arrival_time = match.group(4)

    simulation_name = f"Parte {parte} - Caso {caso}"

    if caso == "2" and inter_arrival_time:
        # Reemplazar '_' por '.' para obtener un número decimal si existe
        inter_arrival_time_formatted = inter_arrival_time.replace("_", ".")
        simulation_name += f" - interArrivalTime: {inter_arrival_time_formatted}"

    return simulation_name

# print(get_simulation_name())

In [ ]:
def get_path_png_with_prefix(prefix: str):
    match = re.search(
        r"Parte (\d)\/p(\d)_c(\d)(?:-(\d(?:_\d+)?))?\.vec", FILE_VEC)

    parte = match.group(1)
    caso = match.group(3)
    inter_arrival_time = match.group(4)

    file_path = DATA_FOLDER + GRAPHICS_FOLDER
    if parte == "1":
        file_path += P1_FOLDER
    else:
        file_path += P2_FOLDER

    file_path += f"{prefix}_p{parte}_c{caso}"

    if caso == "2" and inter_arrival_time:
        file_path += f"-{inter_arrival_time}"

    file_path += ".png"

    return file_path


# print(get_path_png_with_prefix("buffer"))
# print(get_path_png_with_prefix("delay"))
# print(get_path_png_with_prefix("linkuse"))
# print(get_path_png_with_prefix("maxbuffer"))

# Métrica Cantidad de Paquetes vs Delay

In [ ]:
nodo_5_app = datos_nodos["Nodo 5"]["App"]

# Filtrar solo las claves que siguen el patrón "Delay Source X"
delay_sources = {k: v for k, v in nodo_5_app.items(
) if re.match(r"Delay Source \d+", k)}

plt.figure(figsize=(15, 7))

for idx, (key, record) in enumerate(delay_sources.items()):
    valores_delay = record[RECORD_VALUE]

    source_num = re.search(r"\d+", key).group()
    x_vals = [i + 1 for i in range(len(valores_delay))]

    # Encontrar el valor máximo
    idx_max = np.argmax(valores_delay)
    x_max = x_vals[idx_max]
    y_max = valores_delay[idx_max]

    # Colores cíclicos en caso de que haya más sources que colores en la paleta
    color = palette2[idx % len(palette2)]

    # Graficar curva y máximo
    sns.lineplot(x=x_vals, y=valores_delay,
                 label=f"Delay desde Nodo {source_num}", color=color, linewidth=3)
    sns.scatterplot(x=[x_max], y=[y_max], color=color)
    plt.annotate(f"({x_max:.2f}, {y_max:.2f})", (x_max, y_max),
                 textcoords="offset points", xytext=(0, 10), ha='center', color=color,
                 fontsize=10, fontweight='bold')

plt.legend()
plt.title("Cantidad de Paquetes vs Delay",  fontsize=16, fontweight='bold')
# plt.suptitle(get_simulation_name(), fontsize=14)
plt.ylabel("Delay")
plt.xlabel("Paquetes")

if SAVE_GRAPHICS:
    plt.savefig(get_path_png_with_prefix("delay"))

plt.show()

# Máximo uso de buffer de cada nodo por link

In [ ]:
nodos = []
links_size = {"Lnk 0": [], "Lnk 1": []}

nodos_ord = sorted(datos_nodos.keys(), key=lambda x: int(x.split()[1]))
metric = "Buffer Size"

for nodo in nodos_ord:
    nodos.append(nodo)

    for link in links_size.keys():
        if not link in datos_nodos[nodo]:
            links_size[link].append(0)
            continue

        if (not metric in datos_nodos[nodo][link]) or len(datos_nodos[nodo][link][metric][RECORD_VALUE]) == 0:
            links_size[link].append(0)
            continue

        max_buffer = int(max(datos_nodos[nodo][link][metric][RECORD_VALUE]))
        links_size[link].append(max_buffer)


x = np.arange(len(nodos))  # Las ubicaciones de los grupos en el eje x
width = 0.35  # Ancho de las barras

plt.figure(figsize=(15, 7))

# Barras para link 0
rects1 = plt.bar(x - width/2, links_size["Lnk 0"], width, label='Link 0', color=palette1[5])

# Barras para link 1
rects2 = plt.bar(x + width/2, links_size['Lnk 1'], width, label='Link 1', color=palette1[0])

plt.ylabel('Tamaño de Buffer')
plt.title('Máximo tamaño de Buffer por Nodo y Link',  fontsize=16, fontweight='bold')
# plt.suptitle(get_simulation_name(), fontsize=14)
# Establece las etiquetas del eje x en el centro de cada grupo
plt.xticks(x, nodos)
plt.xlabel('Nodo')
plt.legend()

plt.tight_layout()

if SAVE_GRAPHICS:
    plt.savefig(get_path_png_with_prefix("maxbuffer"))
plt.show()

# Tamaño del buffer por link a lo largo del tiempo

In [ ]:
plt.figure(figsize=(15, 7))
nodos_ord = sorted(datos_nodos.keys(), key=lambda x: int(x.split()[1]))
line_styles = ['-', '--', ':', '-.'] # Diferentes estilos de línea para variar si hay muchas curvas

for i, node in enumerate(nodos_ord):
    node_data = datos_nodos[node]
    colors = palette2[i % len(palette2)] # Asigna un color de la paleta, ciclando si hay más nodos que colores

    for link_type in ["Lnk 0", "Lnk 1"]:
        if link_type in node_data and "Buffer Size" in node_data[link_type]:
            buffer_records = node_data[link_type]["Buffer Size"]

            times = buffer_records[RECORD_TIME]
            values = buffer_records[RECORD_VALUE]

            if len(times) > 0 and len(values) > 0:
                label = f"Nodo {i} - {link_type}"
                plt.plot(times, values, label=label, color=colors,
                            linestyle=line_styles[(i * 2 + (0 if link_type == "Lnk 0" else 1)) % len(line_styles)])


plt.title("Tamaño del Búfer de los Links a lo largo del Tiempo",
          fontsize=16, fontweight='bold')
# plt.suptitle(get_simulation_name(), fontsize=14)
plt.xlabel("Tiempo de Simulación")
plt.ylabel("Tamaño del Búfer")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.grid(True)
plt.tight_layout()

if SAVE_GRAPHICS:
    plt.savefig(get_path_png_with_prefix("buffer"))

plt.show()

# Uso de links por nodo

In [ ]:
nodos = []
links_use = {"Lnk 0": [], "Lnk 1": []}

nodos_ord = sorted(datos_nodos.keys(), key=lambda x: int(x.split()[1]))
metric = "Lnk use"

for nodo in nodos_ord:
    nodos.append(nodo)

    for link in links_use.keys():
        if not link in datos_nodos[nodo]:
            links_use[link].append(0)
            continue

        if (not metric in datos_nodos[nodo][link]) or len(datos_nodos[nodo][link][metric][RECORD_VALUE]) == 0:
            links_use[link].append(0)
            continue

        link_use = len(datos_nodos[nodo][link][metric][RECORD_VALUE])
        links_use[link].append(link_use)


# Las ubicaciones de los grupos en el eje x
x = np.arange(len(nodos))
width = 0.35  # Ancho de las barras

plt.figure(figsize=(15, 7))

# Barras para link 0
rects1 = plt.bar(
    x - width/2, links_use["Lnk 0"], width, label='Link 0', color=palette1[5])

# Barras para link 1
rects2 = plt.bar(
    x + width/2, links_use['Lnk 1'], width, label='Link 1', color=palette1[0])

plt.ylabel('Uso de los links')
plt.title('Uso de links por Nodo',
          fontsize=16, fontweight='bold')
# plt.suptitle(get_simulation_name(), fontsize=14)
# Establece las etiquetas del eje x en el centro de cada grupo
plt.xticks(x, nodos)
plt.xlabel('Nodo')
plt.legend()

plt.tight_layout()

if SAVE_GRAPHICS:
    plt.savefig(get_path_png_with_prefix("linkuse"))

plt.show()